In [1]:
!pip install psycopg2

In [2]:
import time

import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Database connection parameters
db_host = "pg_container"
db_name = "test_db"
db_name2 = "example_db"
db_user = "shoc"
db_password = "password"

In [3]:
#conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)
conn=psycopg2.connect( 
    database=db_name, 
    user=db_user, 
    password=db_password, 
    host=db_host, 
    port="5432"
) 

#conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
conn.autocommit = True
cur = conn.cursor()
query = f"CREATE DATABASE {db_name2}"
cur.execute(query)
conn.close()

In [13]:
conn2=psycopg2.connect( 
    database=db_name2, 
    user=db_user, 
    password=db_password, 
    host=db_host, 
    port="5432"
) 
cur = conn2.cursor()


In [14]:
query = """
-- Table with CHAR
CREATE TABLE CharTable (
    ID SERIAL PRIMARY KEY,
    Name CHAR(50),
    OtherData TEXT -- Add other relevant columns
);

-- Table with VARCHAR
CREATE TABLE VarcharTable (
    ID SERIAL PRIMARY KEY,
    Name VARCHAR(50),
    OtherData TEXT -- Add other relevant columns
);

-- Insert data (ensure data is less than 50 chars for 'Name')
INSERT INTO CharTable (Name, OtherData)
SELECT md5(random()::text), 'Some additional text'
FROM generate_series(1, 100000); -- 100,000 rows

INSERT INTO VarcharTable (Name, OtherData)
SELECT Name, OtherData FROM CharTable; -- copy data from CharTable

-- Optionally, add an index on the Name column (after the data is loaded)
-- CREATE INDEX idx_char_name ON CharTable (Name);
-- CREATE INDEX idx_varchar_name ON VarcharTable (Name);
"""

cur.execute(query)

In [16]:
import pandas as pd

query = [
    "EXPLAIN ANALYZE SELECT * FROM CharTable;", 
    "EXPLAIN ANALYZE SELECT * FROM VarcharTable;", 
    "EXPLAIN ANALYZE SELECT * FROM CharTable WHERE TRIM(Name) = 'your_test_string';", 
    "EXPLAIN ANALYZE SELECT * FROM VarcharTable WHERE Name = 'your_test_string';", 
    "EXPLAIN ANALYZE SELECT COUNT(*) FROM CharTable WHERE OtherData LIKE '%text%';", 
    "EXPLAIN ANALYZE SELECT COUNT(*) FROM VarcharTable WHERE OtherData LIKE '%text%';", 
    "EXPLAIN ANALYZE SELECT * FROM CharTable ORDER BY TRIM(Name);", 
    "EXPLAIN ANALYZE SELECT * FROM VarcharTable ORDER BY Name;",
]

df = {
    "query": [],
    "col1": [],
    "col2": [],
    "col3": [],
}

for q in query:
    cur.execute(q)
    rows = cur.fetchall()

    # print(f"Results of '{q}':")
    # print(rows)
    # print(" ================================ ")

    df["query"].append(q)
    df["col1"].append(rows[0])
    df["col2"].append(rows[1])
    df["col3"].append(rows[2])

conn2.commit()
cur.close()
conn2.close()

df = pd.DataFrame(df)
df

,query,col1,col2,col3
0,EXPLAIN ANALYZE SELECT * FROM CharTable;,(Seq Scan on chartable (cost=0.00..1734.20 ro...,"(Planning Time: 0.039 ms,)","(Execution Time: 20.112 ms,)"
1,EXPLAIN ANALYZE SELECT * FROM VarcharTable;,(Seq Scan on varchartable (cost=0.00..1637.28...,"(Planning Time: 0.026 ms,)","(Execution Time: 16.903 ms,)"
2,EXPLAIN ANALYZE SELECT * FROM CharTable WHERE ...,(Seq Scan on chartable (cost=0.00..2034.35 ro...,( Filter: (TRIM(BOTH FROM name) = 'your_test_...,"( Rows Removed by Filter: 100000,)"
3,EXPLAIN ANALYZE SELECT * FROM VarcharTable WHE...,(Seq Scan on varchartable (cost=0.00..1762.35...,( Filter: ((name)::text = 'your_test_string':...,"( Rows Removed by Filter: 100000,)"
4,EXPLAIN ANALYZE SELECT COUNT(*) FROM CharTable...,(Aggregate (cost=1835.05..1835.06 rows=1 widt...,( -> Seq Scan on chartable (cost=0.00..1834...,"( Filter: (otherdata ~~ '%text%'::text),)"
5,EXPLAIN ANALYZE SELECT COUNT(*) FROM VarcharTa...,(Aggregate (cost=1763.35..1763.36 rows=1 widt...,( -> Seq Scan on varchartable (cost=0.00..1...,"( Filter: (otherdata ~~ '%text%'::text),)"
6,EXPLAIN ANALYZE SELECT * FROM CharTable ORDER ...,(Gather Merge (cost=7375.04..10082.26 rows=23...,"( Workers Planned: 1,)","( Workers Launched: 1,)"
7,EXPLAIN ANALYZE SELECT * FROM VarcharTable ORD...,(Sort (cost=9476.08..9601.15 rows=50028 width...,"( Sort Key: name,)","( Sort Method: external merge Disk: 6664kB,)"


In [17]:
import psycopg2
import time


def execute_query(query):
    conn = None
    try:
        conn = psycopg2.connect(host=db_host, database=db_name2, user=db_user, password=db_password)
        cur = conn.cursor()

        start_time = time.time()
        cur.execute(query)
        rows = cur.fetchall()
        end_time = time.time()

        elapsed_time = end_time - start_time
        return elapsed_time

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn:
            cur.close()
            conn.close()

# Define your test queries
char_query = "SELECT * FROM CharTable WHERE TRIM(Name) = 'your_test_string';"
varchar_query = "SELECT * FROM VarcharTable WHERE Name = 'your_test_string';"

# Number of runs
num_runs = 10

# Run the tests and time them
char_times = []
varchar_times = []

for _ in range(num_runs):
    char_times.append(execute_query(char_query))
    varchar_times.append(execute_query(varchar_query))

# Calculate averages (excluding the first run)
avg_char_time = sum(char_times[1:]) / (num_runs - 1)
avg_varchar_time = sum(varchar_times[1:]) / (num_runs - 1)

print(f"Average CHAR query time: {avg_char_time:.6f} seconds")
print(f"Average VARCHAR query time: {avg_varchar_time:.6f} seconds")

Average CHAR query time: 0.088747 seconds
Average VARCHAR query time: 0.010051 seconds


In [18]:
conn=psycopg2.connect( 
    database=db_name, 
    user=db_user, 
    password=db_password, 
    host=db_host, 
    port="5432"
) 

#conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
conn.autocommit = True
cur = conn.cursor()

query = f"DROP DATABASE {db_name2}"
cur.execute(query)
conn.close()

ObjectInUse: database "example_db" is being accessed by other users
DETAIL:  There are 2 other sessions using the database.
